In [ ]:
USE TSQL2012;

# Non-correlated Subqueries

When the internal query can run independent of the outer query.

In [ ]:
-- scalar subqueries
-- products with a minimum price
SELECT productid, productname, unitprice
FROM Production.Products
WHERE unitprice =
  (SELECT MIN(unitprice)
   FROM Production.Products);

In [ ]:
-- multi-valued subqieries
-- products supplied by suppliers from Japan
SELECT productid, productname, unitprice
FROM Production.Products
WHERE supplierid IN
  (SELECT supplierid
   FROM Production.Suppliers
   WHERE country = N'Japan');

# Correlated Subqueries
When the internal query can NOT run independent of the outer query.

In [ ]:
-- products with minimum unitprice per each category
-- Logic: at each iteration the min price of the each catergory from the above query is computed.
SELECT categoryid, productid, productname, unitprice
FROM Production.Products AS P1
WHERE unitprice =
  (SELECT MIN(unitprice)
   FROM Production.Products AS P2
   WHERE P2.categoryid = P1.categoryid);

## Exercise 1
Get all customers who has placed an order on February 12, 2007. (Use `Sales.Customers` and `Sales.Orders` tables)

Hint: Use `WHERE EXISTS` instead of `WHERE`.
```
SELECT custid, contactname
FROM Sales.Customers AS C
WHERE EXISTS (
    SELECT *
   FROM Sales.Orders ...
)
```

## Exercise 2

Get all customers who has NOT placed an order on February 12, 2007. Can you guess how to modify query of above exercise?

# Partition by

In [ ]:
-- row numbers for products
-- partitioned by categoryid, ordered by unitprice, productid
SELECT
  ROW_NUMBER() OVER(PARTITION BY categoryid
                    ORDER BY unitprice, productid) AS rownum,
  categoryid, productid, productname, unitprice
FROM Production.Products;

## Exercise 3
Use the above query to two products with lowest prices per each category.

# Derived Tables: WITH clause

In [ ]:
-- two products with lowest prices per category
WITH C AS
(
  SELECT
    ROW_NUMBER() OVER(PARTITION BY categoryid
                      ORDER BY unitprice, productid) AS rownum,
    categoryid, productid, productname, unitprice
  FROM Production.Products
)
SELECT categoryid, productid, productname, unitprice
FROM C
WHERE rownum <= 2;

# Bonus: Recursive CTE

In [ ]:
-- For further studies
-- Recursive CTE
-- management chain leading to given employee
WITH EmpsCTE AS
(
  SELECT empid, mgrid, firstname, lastname, 0 AS distance
  FROM HR.Employees
  WHERE empid = 9

  UNION ALL

  SELECT M.empid, M.mgrid, M.firstname, M.lastname, S.distance + 1 AS distance
  FROM EmpsCTE AS S
    JOIN HR.Employees AS M
      ON S.mgrid = M.empid
)
SELECT empid, mgrid, firstname, lastname, distance
FROM EmpsCTE;